In [22]:
import sys
import pathlib
import numpy as np
import pandas as pd
from rich.progress import track
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

sys.path.append("../src")
from utils.feature import FeatureStore
from utils.common import load_pickle, dump_pickle, save_cache

In [23]:
from dataset import IndoorDataset, IndoorDataModule
from submission import IndoorTestDataset

In [25]:
d1 = torch.zeros((10))
d2 = torch.ones((10)) * 2

print(d1)
print(d2)

loss_fn = nn.MSELoss()
loss_fn(d1, d2)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])


tensor(4.)

In [20]:
torch.mean(torch.pow(torch.abs(d1 - d2), 2))

tensor(4.)

In [82]:
d1 = torch.tensor(
    [
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
    ], dtype=torch.float)
d2 = torch.tensor(
    [
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
    ], dtype=torch.float)

print(d1)
print(d2)

loss_fn = nn.MSELoss()
loss_fn(d1, d2)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.]])


tensor(8.5000)

In [83]:
diff = (d1 - d2)
diff

tensor([[-1., -4.],
        [-1., -4.],
        [-1., -4.],
        [-1., -4.],
        [-1., -4.],
        [-1., -4.],
        [-1., -4.],
        [-1., -4.],
        [-1., -4.],
        [-1., -4.]])

In [84]:
tmp = torch.pow(diff, 2)
tmp

tensor([[ 1., 16.],
        [ 1., 16.],
        [ 1., 16.],
        [ 1., 16.],
        [ 1., 16.],
        [ 1., 16.],
        [ 1., 16.],
        [ 1., 16.],
        [ 1., 16.],
        [ 1., 16.]])

In [85]:
tmp = torch.sqrt(tmp)
tmp

tensor([[1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.]])

In [86]:
tmp = torch.sum(tmp, dim=1)
tmp

tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [87]:
torch.mean(tmp)

tensor(5.)

In [114]:
class MSELoss(nn.Module):
    def __init__(self):
        super(MSELoss, self).__init__()

    def forward(self, y_hat, y):
        pos_error = y_hat - y
        pos_error = torch.sum(torch.abs(pos_error), dim=1)
        return torch.mean(pos_error)

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, y_hat, y):
        pos_error = y_hat - y
        pos_error = torch.sum(torch.sqrt(torch.pow(pos_error, 2)), dim=1)
        return torch.mean(pos_error)


class MeanPositionLoss(nn.Module):
    def __init__(self):
        super(MeanPositionLoss, self).__init__()

    def forward(self, y_hat, y, floor_hat, floor):
        pos_error = y_hat - y
        pos_error = torch.sum(torch.sqrt(torch.pow(pos_error, 2)), dim=1)
        
        p = 15
        floor_error = p * torch.abs(floor_hat - floor)
        
        error = pos_error + floor_error
        return torch.mean(error)

In [115]:
# class MeanPositionLoss(nn.Module):
#     def __init__(self):
#         super(MeanPositionLoss, self).__init__()

#     def forward(self, y_hat, y):
#         pos = y
#         pos_hat = y_hat

#         diff_x = pos_hat[:, 0] - pos[:, 0]
#         diff_y = pos_hat[:, 1] - pos[:, 1]

#         error = torch.sqrt(torch.pow(diff_x, 2) + torch.pow(diff_y, 2))
#         return torch.mean(error)

In [116]:
loss_fn = MeanPositionLoss()
loss_fn(d1, d2, torch.zeros(10), torch.zeros(10))

tensor(5.)

In [117]:
loss_fn = RMSELoss()
loss_fn(d1, d2)

tensor(5.)

In [118]:
loss_fn = MSELoss()
loss_fn(d1, d2)

tensor(5.)

In [69]:
result = load_pickle("../data/preprocessing/train_wifi_results.pkl")

Load pickle from ../data/preprocessing/train_wifi_results.pkl


In [74]:
bssid, rssi, freq, last_seen_ts = zip(*result)

In [79]:
bssid = np.concatenate(bssid, axis=0).astype("int32")
rssi = np.concatenate(rssi, axis=0)
freq = np.concatenate(freq, axis=0)
last_seen_ts = np.concatenate(last_seen_ts, axis=0)

In [87]:
np.unique(bssid[:, 2]).shape

(46866,)

In [61]:
dataset = IndoorTestDataset()
dataloader = DataLoader(
    dataset,
    batch_size=64,
    num_workers=8,
    shuffle=False,
    drop_last=False,
)

Load pickle from ../data/submit/test_waypoint.pkl


In [62]:
next(iter(dataloader))

[[tensor([[1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1]]),
  tensor([[0],
          [0],
        

In [60]:
n_fold = 0

train_idx = np.load(f"../data/fold/fold{n_fold:>02}_train_idx.npy")
valid_idx = np.load(f"../data/fold/fold{n_fold:>02}_valid_idx.npy")
test_idx = np.load(f"../data/fold/fold{n_fold:>02}_test_idx.npy")

# Define and setup datamodule.
datamodule = IndoorDataModule(64, train_idx, valid_idx, test_idx)
datamodule.setup()

In [63]:
trainloader = datamodule.test_dataloader()

In [64]:
next(iter(trainloader))

[[[tensor([[181],
           [ 59],
           [ 26],
           [ 16],
           [ 75],
           [119],
           [ 84],
           [203],
           [ 46],
           [204],
           [157],
           [ 90],
           [200],
           [183],
           [202],
           [201],
           [199],
           [117],
           [181],
           [ 95],
           [180],
           [194],
           [ 95],
           [191],
           [113],
           [200],
           [181],
           [ 92],
           [113],
           [134],
           [ 10],
           [ 85],
           [ 72],
           [199],
           [118],
           [ 51],
           [181],
           [190],
           [181],
           [182],
           [181],
           [201],
           [ 60],
           [ 84],
           [194],
           [117],
           [187],
           [103],
           [203],
           [127],
           [ 84],
           [ 52],
           [198],
           [190],
           [190],
          